In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("./data/train_dataset.csv")

data['is_test'] = False
evaluation = pd.read_csv("./data/evaluation_public.csv")
evaluation['is_test'] = True
sample = pd.read_csv("./data/sample_submission.csv")

all_data = pd.concat([data, evaluation]).reset_index(drop=True)

# 特征处理

In [3]:
all_data['date'] = pd.to_datetime(all_data['time'])
all_data['hour'] = all_data['date'].dt.hour
all_data['year'] = all_data['date'].dt.year
all_data['month'] = all_data['date'].dt.month
all_data['minute'] = all_data['date'].dt.minute
all_data['weekday'] = all_data['date'].dt.weekday
all_data['day'] = all_data['date'].dt.day
all_data['hour'] = all_data['date'].dt.hour
all_data['ts'] = all_data['hour']*3600 + all_data['minute']*60 + all_data['date'].dt.second

In [4]:
import gc
features = [ 'JS_NH3', 'CS_NH3', 'JS_TN', 'CS_TN', 'JS_LL', 'CS_LL', 'MCCS_NH4', 'MCCS_NO3', 'JS_COD', 'CS_COD', 'JS_SW', 'CS_SW', 'B_HYC_NH4', 'B_HYC_XD', 'B_HYC_MLSS', 'B_HYC_JS_DO', 'B_HYC_DO', 'B_CS_MQ_SSLL', 'B_QY_ORP', 'N_HYC_NH4', 'N_HYC_XD', 'N_HYC_MLSS', 'N_HYC_JS_DO', 'N_HYC_DO', 'N_CS_MQ_SSLL', 'N_QY_ORP','weekday','hour', 'ts']
features = [f for f in features if f not in ['time', 'Label1', 'Label2','Label1_log','Label2_log','CS_LL',
                                             'CS_NH3', 'JS_SW'
                                               # 'B_QY_ORP','JS_TN', 'CS_SW','MCCS_NH4','N_HYC_JS_DO','MCCS_NO3','JS_SW',
                                             ]]

labels = ['Label1', 'Label2']
train = all_data[~all_data['is_test']].copy(deep=True)
test = all_data[all_data['is_test']].copy(deep=True)
# train = train.dropna(subset=['Label1', 'Label2']).reset_index(drop=True)
test['is_train'] = False
train['is_train'] = True
data = pd.concat([train, test]).reset_index(drop=True)
del all_data
gc.collect()

42

# 通过训练找出在训练集和测试集中差别很大的样本

In [5]:
from sklearn.model_selection import train_test_split
train['is_test'] = 0
test['is_test'] = 1
data_v2 = pd.concat([train, test]).reset_index(drop=True)
for f in features:
    if f not in ['weekday','hour', 'ts']:
        q = len(data_v2[f].drop_duplicates())
        data_v2[f] = pd.qcut(data_v2[f], q=int(q/10), labels=False, duplicates="drop")

In [26]:
import lightgbm as lgb
# N_HYC_NH4
for f in [ 'JS_NH3', 'CS_NH3', 'JS_TN', 'CS_TN', 'JS_LL', 'MCCS_NH4', 'MCCS_NO3', 'JS_COD', 'CS_COD', 'JS_SW', 'CS_SW', 'B_HYC_NH4', 'B_HYC_XD', 'B_HYC_MLSS', 'B_HYC_JS_DO', 'B_HYC_DO', 'B_CS_MQ_SSLL', 'B_QY_ORP', 'N_HYC_NH4', 'N_HYC_XD', 'N_HYC_MLSS', 'N_HYC_JS_DO', 'N_HYC_DO', 'N_CS_MQ_SSLL']:
    features_filter = ['ts', 'weekday', f]
    print(f)
    X_train, X_test, y_train, y_test  = train_test_split(data_v2[features_filter], data_v2['is_test'], test_size=0.2)
    model = lgb.LGBMClassifier(
            boosting="gbdt",
            max_depth=4,
            learning_rate=0.005,
            n_estimators=500,
            min_child_weight=1,
            min_data_in_leaf=60,
            subsample = 0.7,
            feature_fraction=0.4,
            bagging_seed=1,
            reg_alpha=0.11,
            reg_lambda=0.1,  # 此处不改了
            min_sum_hessian_in_leaf=0.01,
            random_state=1212
        )
    model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_test, y_test)], eval_metric=['auc'],
              early_stopping_rounds=20, verbose=200)
    print("\n")


JS_NH3
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[1]	training's auc: 0.505011	training's binary_logloss: 0.244544	valid_1's auc: 0.487318	valid_1's binary_logloss: 0.243667


CS_NH3
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[1]	training's auc: 0.504178	training's binary_logloss: 0.244522	valid_1's auc: 0.490698	valid_1's binary_logloss: 0.243754


JS_TN
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[1]	training's auc: 0.502673	training's binary_logloss: 0.244851	valid_1's auc: 0.496647	valid_1's binary_logloss: 0.242439


CS_TN
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[1]	training's auc: 0.50293	training's binary_logloss: 0.245727	valid_1's auc: 0.495456	valid_1's binary_logloss: 0.238944


JS_LL
Training until validation scores don't improve for 20 rounds
Early stopping, best